##  Más de la mitad de los partos en México en 2024 fueron cesáreas


**Este proyecto utiliza la base de datos del Sistema Nacional de Información de Certificados de Nacimientos (SINAC) 2024 de la Secretaría de Salud para analizar la preponderancia de cesáreas en el contexto de la saturación obstétrica en México.** 
**El análisis se centra en:**
- Cuantificar la proporción de cesáreas versus partos vaginales.
- Identificar variaciones por entidad federativa y por hospital.
- Evaluar la disponibilidad de gineco-obstetras en los partos registrados.
- Reflexionar sobre las implicaciones para la seguridad materna y la emergencia obstétrica.

## Objetivos
1. Calcular la tasa nacional de cesáreas y compararla con los partos vaginales.  
2. Mapear la distribución de cesáreas por estado y hospital.  
3. Detectar hospitales con alto número de partos sin gineco-obstetra.  
4. Generar insumos para debate sobre políticas de salud materna en México.

### Librerías:

In [2231]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl

### Configuración del notebook:

In [2232]:
pd.set_option('display.max_columns', None)

### Diccionarios

**Diccionario 'orden_columnas' será util para aplicar un orden final al df generado.**

In [2233]:
orden_columnas = [
    'CLUES',
    'ENTIDAD_PARTO_CVE',
    'ENTIDAD_PARTO_STR',
    'TIPO_PARTO_CVE',
    'TIPO_PARTO_STR',
    'TIPO_CESAREA_CVE',
    'TIPO_CESAREA_STR',
    'PERSONAL_ATENDIO_CVE',
    'PERSONA_ATENDIO_STR',
    'MEDICO_ATENDIO_CVE',
    'MEDICO_ATENDIO_STR',
    'SOBREVIVIO_PARTO_CVE',
    'SOBREVIVIO_PARTO_STR',
    'FECHA_NACIMIENTO_MADRE'
]


**Diccionario 'medico_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [2234]:
medico_map = {
    11: 'MÉDICO GINECO OBSTETRA',
    12: 'OTRO ESPECIALISTA',
    13: 'RESIDENTE',
    14: 'MÉDICO GENERAL',
    15: 'MPSS',
    16: 'MIP',
    9:  'SE IGNORA'
}

**Diccionario 'persona_atendio_parto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [2235]:
personal_atendio_parto_map = {
    0: 'NO ESPECIFICADO',
    1: 'MÉDICO',
    2: 'ENFERMERA',
    3: 'PERSONA AUTORIZADA POR LA SECRETARÍA DE SALUD',
    4: 'PARTERA',
    8: 'OTRO',
    9: 'SE IGNORA' 
}

**Diccionario 'parto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [2236]:
parto_map = {
    0: 'No especificado',
    1: 'Eutócico (natural)',
    2: 'Distócico (natural complicado)',
    3: 'Cesárea',
    8: 'Otro'
}

**Diccionario 'cesareas_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [2237]:
cesareas_map = {
    0: "NO ESPECIFICADO",
    1: "PROGRAMADA",
    2: "DE URGENCIA",
    8: "NO APLICA"
}

**Diccionario 'entidadfederativaparto_map' será util para añadir una columna _str en las columnas que de origen contienen claves.**

In [2238]:
entidad_federativaparto_map = {
    0: 'NO ESPECIFICADO',
    1: 'AGUASCALIENTES',
    2: 'BAJA CALIFORNIA',
    3: 'BAJA CALIFORNIA SUR',
    4: 'CAMPECHE',
    5: 'COAHUILA DE ZARAGOZA',
    6: 'COLIMA',
    7: 'CHIAPAS',
    8: 'CHIHUAHUA',
    9: 'CIUDAD DE MÉXICO',
    10: 'DURANGO',
    11: 'GUANAJUATO',
    12: 'GUERRERO',
    13: 'HIDALGO',
    14: 'JALISCO',
    15: 'MÉXICO',
    16: 'MICHOACÁN DE OCAMPO',
    17: 'MORELOS',
    18: 'NAYARIT',
    19: 'NUEVO LEÓN',
    20: 'OAXACA',
    21: 'PUEBLA',
    22: 'QUERÉTARO',
    23: 'QUINTANA ROO',
    24: 'SAN LUIS POTOSÍ',
    25: 'SINALOA',
    26: 'SONORA',
    27: 'TABASCO',
    28: 'TAMAULIPAS',
    29: 'TLAXCALA',
    30: 'VERACRUZ DE IGNACIO DE LA LLAVE',
    31: 'YUCATÁN',
    32: 'ZACATECAS',
    88: 'NO APLICA',
    99: 'SE IGNORA'
}

**Diccionario para homologar nombres de entidades***

In [2239]:
reemplazos_entidades = {
    "CIUDAD DE MÉXICO": "Ciudad de México",
    "VERACRUZ DE IGNACIO DE LA LLAVE": "Veracruz",
    "MICHOACÁN DE OCAMPO": "Michoacán",
    "COAHUILA DE ZARAGOZA": "Coahuila",
    "BAJA CALIFORNIA SUR": "Baja California Sur",
    "BAJA CALIFORNIA": "Baja California",
    "NUEVO LEÓN": "Nuevo Leon",
    "QUERÉTARO": "Queretaro",
    "SAN LUIS POTOSÍ": "San Luis Potosi",
    "TLAXCALA": "Tlaxcala",
    "SONORA": "Sonora",
    "YUCATÁN": "Yucatan",
    "CAMPECHE": "Campeche",
    "DURANGO": "Durango",
    "ZACATECAS": "Zacatecas",
    "HIDALGO": "Hidalgo",
    "AGUASCALIENTES": "Aguascalientes",
    "MORELOS": "Morelos",
    "QUINTANA ROO": "Quintana Roo",
    "TABASCO": "Tabasco",
    "CHIAPAS": "Chiapas",
    "GUERRERO": "Guerrero",
    "OAXACA": "Oaxaca",
    "NAYARIT": "Nayarit",
    "COLIMA": "Colima",
    "SINALOA": "Sinaloa",
    "JALISCO": "Jalisco",
    "PUEBLA": "Puebla",
    "TAMAULIPAS": "Tamaulipas",
    "CHIHUAHUA": "Chihuahua",
    "ESTADO DE MÉXICO": "Mexico"
}

**Diccionario para identificar si la madre sobrevivió o no al parto (mortalidad materna)**


In [2240]:
mortalidad_materna_map = {
    0: "NO ESPECIFICADO",
    1: "SI",
    2: "NO",
    8: "NO APLICA",
    9: "SE IGNORA"
}


### Columnas útiles

In [2241]:
columnas_nacimientos = [
    'CLUES',
    'ENTIDADFEDERATIVAPARTO',
    'RESOLUCIONEMBARAZO',
    'TIPOCESAREA',
    'PERSONALATENDIO',
    'TIPOMEDICOATENDIO',
    'SOBREVIVIOPARTO',
    'FECHANACIMIENTOMADRE'
]

In [2242]:
columnas_establecimientos_salud = [
    'CLUES',
    'NOMBRE DE LA UNIDAD',
    'LATITUD',
    'LONGITUD',
    'ENTIDAD',
    'CLAVE DE LA ENTIDAD',
    'NOMBRE DE LA INS ADM',
    'NIVEL ATENCION',
    'NOMBRE DE LA INSTITUCION',
    'NOMBRE COMERCIAL',
]


### Cargas:

**Lectura del dataset de nacimientos 2024 proporcionado por Secretaría de Salud**

In [2243]:
df_nacimientos_2024 = pd.read_csv("Nacimientos_2024.csv", encoding='utf-8', low_memory=False, usecols=columnas_nacimientos)


**Lectura del catálogo de establecimientos de salud**

In [2244]:
df_establecimientos_salud_2024 = pd.read_excel('sinac_catalogos_2024/ESTABLECIMIENTOS_SALUD.xlsx', usecols=columnas_establecimientos_salud)

# Ejecución: 

## RENOMBRE DE COLUMNAS

In [2245]:
#Renombrar las columnas df_nacimientos_2024

df_nacimientos_2024 = df_nacimientos_2024.rename(columns={
    'FECHANACIMIENTOMADRE':'FECHA_NACIMIENTO_MADRE',
    'SOBREVIVIOPARTO': 'SOBREVIVIO_PARTO_CVE',
    'CLUES':'CLUES',
    'RESOLUCIONEMBARAZO' : 'TIPO_PARTO_CVE',
    'TIPOCESAREA': 'TIPO_CESAREA_CVE',
    'PERSONALATENDIO':'PERSONAL_ATENDIO_CVE',
    'TIPOMEDICOATENDIO':'MEDICO_ATENDIO_CVE',
    'ENTIDADFEDERATIVAPARTO': 'ENTIDAD_PARTO_CVE',
})

## TRANSFORMACIÓN DE CVES A STR

**Transformación de nombre en columna TIPOMEDICOATENDIO a MEDICO_ATENDIO_STR**

In [2246]:
df_nacimientos_2024['MEDICO_ATENDIO_STR'] = df_nacimientos_2024['MEDICO_ATENDIO_CVE'].map(medico_map)

**Transformación de nombre en columna PERSONALATENDIO a PERSONAL_ATENDIO_STR**

In [2247]:
df_nacimientos_2024['PERSONA_ATENDIO_STR'] = df_nacimientos_2024['PERSONAL_ATENDIO_CVE'].map(personal_atendio_parto_map)

**Transformación de nombre en columna RESOLUCIONEMBARAZO a TIPO_PARTO_STR**

In [2248]:
df_nacimientos_2024['TIPO_PARTO_STR'] = df_nacimientos_2024['TIPO_PARTO_CVE'].map(parto_map)

**Transformación de nombre en columna ENTIDADFEDERATIVAPARTO a ENTIDAD_PARTO_STR**

In [2249]:
df_nacimientos_2024['ENTIDAD_PARTO_STR'] = df_nacimientos_2024['ENTIDAD_PARTO_CVE'].map(entidad_federativaparto_map)

**Transformación de nombre en columna SOBREVIVIOPARTO a SOBREVIVIO_PARTO_CVE**

In [2250]:
df_nacimientos_2024['SOBREVIVIO_PARTO_STR'] = df_nacimientos_2024['SOBREVIVIO_PARTO_CVE'].map(mortalidad_materna_map)

**Transformación de nombre en columna TIPO_CESAREA_CVE a TIPO_CESAREA_STR**

In [2251]:
df_nacimientos_2024['TIPO_CESAREA_STR'] = df_nacimientos_2024['TIPO_CESAREA_CVE'].map(cesareas_map)

## ORDENAR COLUMNAS

In [2252]:
df_nacimientos_2024 = df_nacimientos_2024[orden_columnas]

## Nota metodológica

#### El conjunto de datos original registró un total de 1 413 203 nacimientos. Primero se eliminaron los 28 registros nulos (NaN) en la columna CLUES, quedando 1 413 175. Después, al excluir los registros con valor '9998', quedaron 1 338 905 nacimientos con información completa sobre la unidad médica.

## Separación de dataframe __df_nacimientos_2024__, según los datos que tiene disponibles

**DataFrame de nacimientos 2024 íntegro sin modificaciones** (copia la referencia (en pandas esto no duplica memoria), y muestra el total de filas iniciales)

In [2253]:
# 1) Partimos de tu DataFrame “base” (antes de limpiar nada):
df_nacimientos_2024 = df_nacimientos_2024
len(df_nacimientos_2024)  
# → 1 413 203 registros totales

1413203

In [2254]:
df_nacimientos_2024['CLUES'].isna().sum()

np.int64(28)

**DataFrame de nacimientos 2024, sin valores NaN en la columna 'CLUES'**

In [2255]:
df_nacimientos_2024_clean_clues = df_nacimientos_2024.dropna(subset=['CLUES'])
len(df_nacimientos_2024_clean_clues)

1413175

**DataFrame de nacimientos 2024, sin valores 9998 en la columna 'CLUES'**

In [2256]:
# 4) Ahora quitamos también los registros con CLUES == '9998'
#    (código que usas para marcarlas como “sin identificador oficial”)
df_nacimientos_2024_clean_clues = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['CLUES'] != '9998'
]
len(df_nacimientos_2024_clean_clues)
# → 1 338 905 registros

1338905

In [2257]:
# 5) Reordenas las columnas según tu esquema final
df_nacimientos_2024_clean_clues = df_nacimientos_2024_clean_clues[orden_columnas]

## ¿Cuántos nacimientos se registraron en 2024 y cómo se distribuyeron según el tipo de parto?


**Determinar el número de registros en df_nacimientos_2024 que contiene la totalidad de nacimientos en México en el año 2024**

In [2258]:
total_nacimientos = len(df_nacimientos_2024)
print("El total de nacimientos en 2024 fue:",total_nacimientos)

El total de nacimientos en 2024 fue: 1413203


### Cesáreas en México 2024: muy por encima del estándar de la OMS

En 2024, como se muestra a continuación, el 56.1% de los nacimientos fueron por cesárea.  

La OMS recomienda un rango ideal de entre **10% y 15%**, solo en casos médicamente necesarios.

Una tasa tan alta puede indicar **saturación hospitalaria, falta de personal especializado o intervenciones innecesarias**.

**Contar el número de nacimientos por tipo de parto**


In [2259]:
df_distribucion_tipo_parto = (
    df_nacimientos_2024["TIPO_PARTO_STR"]
    .value_counts(dropna=False)                  
    .rename_axis("TIPO_PARTO")
    .reset_index(name="TOTAL")
    .assign(PORCENTAJE=lambda d: (d["TOTAL"] / total_nacimientos * 100).round(1))
    .sort_values("PORCENTAJE", ascending=False)
    .reset_index(drop=True)
)

**Generación .csv de df_distribucion_tipo_parto**

In [2260]:
df_distribucion_tipo_parto.to_csv('tipo_parto_distribucion_2024_plot.csv', index=False)

## Cesáreas por entidad 

1. Se creó una columna binaria `ES_CESAREA` a partir de `TIPO_PARTO_CVE == 3`.
2. Se agruparon los datos por entidad (`ENTIDAD_PARTO_STR`) para contar:
   - El total de partos (`TOTAL_PARTOS`).
   - El total de cesáreas (`TOTAL_CESAREAS`).
3. Se calculó el porcentaje de cesáreas como:
   \[
   \text{PORCENTAJE_CESAREAS} = \left( \frac{\text{TOTAL_CESAREAS}}{\text{TOTAL_PARTOS}} \right) \times 100
   \]
4. Se generó un ranking nacional ordenado de mayor a menor porcentaje.

In [2261]:
df_nacimientos_2024["ES_CESAREA"] = df_nacimientos_2024["TIPO_PARTO_CVE"] == 3

In [2262]:
df_cesareas_por_estado["ENTIDAD_PARTO_STR"] = df_cesareas_por_estado["ENTIDAD_PARTO_STR"].replace(reemplazos_entidades)

In [2263]:
df_cesareas_por_estado["PORCENTAJE_CESAREAS"] = (
    df_cesareas_por_estado["TOTAL_CESAREAS"] / df_cesareas_por_estado["TOTAL_PARTOS"] * 100
).round(1)

In [2264]:
df_cesareas_por_estado = df_cesareas_por_estado.sort_values("PORCENTAJE_CESAREAS", ascending=False).reset_index(drop=True)
df_cesareas_por_estado["RANK"] = df_cesareas_por_estado.index + 1

In [2265]:
df_cesareas_por_estado["RANK"] = df_cesareas_por_estado.index + 1

In [2266]:
df_cesareas_por_estado.head(10)

,ENTIDAD_PARTO_STR,TOTAL_PARTOS,TOTAL_CESAREAS,PORCENTAJE_CESAREAS,RANK
0,Sinaloa,34367,22196,64.6,1
1,Tlaxcala,16042,10171,63.4,2
2,Tamaulipas,40518,25658,63.3,3
3,Puebla,84255,52285,62.1,4
4,Nuevo Leon,70433,43687,62.0,5
5,Michoacán,58070,35908,61.8,6
6,Veracruz,78230,47036,60.1,7
7,Aguascalientes,18512,11058,59.7,8
8,Jalisco,96717,57555,59.5,9
9,Yucatan,24541,14319,58.3,10


In [2267]:
df_export = df_cesareas_por_estado[["ENTIDAD_PARTO_STR", "PORCENTAJE_CESAREAS"]].rename(
    columns={
        "ENTIDAD_PARTO_STR": "Entidad",
        "PORCENTAJE_CESAREAS": "Porcentaje"
    }
)

In [2268]:
df_export.to_csv("porcentaje_cesareas_por_estado_plot.csv", index=False)

In [2269]:
# Suma total nacional
total_partos_nacional = df_cesareas_por_estado["TOTAL_PARTOS"].sum()
total_cesareas_nacional = df_cesareas_por_estado["TOTAL_CESAREAS"].sum()

In [2270]:
print(total_partos_nacional)
print(total_cesareas_nacional)


1413203
792152


## Identificar partos por entidad, hospitales (CLUES) y tipo de cesárea. 

**La base de datos original presenta 28,885 registros en los que la variable MEDIDO_ATENDIO (CVE,STR) aparece como NaN, lo que significa que no se reportó el tipo de médico que atendió esos partos.**

In [2271]:
# Conteo de valores faltantes por columna
missing_counts = df_nacimientos_2024.isna().sum()

# Porcentaje de faltantes por columna
missing_pct = (missing_counts / len(df_nacimientos_2024) * 100).round(2)

# Tabla solo con columnas que tienen al menos un NaN, ordenada de mayor a menor
report = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_pct": missing_pct
}).loc[missing_counts > 0].sort_values("missing_count", ascending=False)

print(report)

                    missing_count  missing_pct
MEDICO_ATENDIO_CVE          28855         2.04
MEDICO_ATENDIO_STR          28855         2.04
CLUES                          28         0.00


In [2272]:
#ORDENAR POR CLUES
df_nacimientos_2024 = df_nacimientos_2024.sort_values(by='CLUES')

In [2273]:
#TEMP
len(df_nacimientos_2024)

1413203

In [2274]:
df_nacimientos_2024.to_csv('dataset_nacimientos_2024.csv', index=False)

## TOP 10 HOSPITALES CON MÁS PARTOS SIN GINECO-OBSTETRAS Y TIPO DE NIVEL. 

In [2275]:
# df_nacimientos_2024_clean_clues

df_top_ranking_hospitales = df_nacimientos_2024_clean_clues.copy()

In [2276]:
#Filtrar los que fueron atentidos por un gineco-obstetra. 
df_top_ranking_hospitales['ES_GINECO'] = df_nacimientos_2024_clean_clues['MEDICO_ATENDIO_STR'].str.contains('MÉDICO GINECO OBSTETRA', case=False, na=False)

In [2277]:
df_top_ranking_hospitales.head()

,CLUES,ENTIDAD_PARTO_CVE,ENTIDAD_PARTO_STR,TIPO_PARTO_CVE,TIPO_PARTO_STR,TIPO_CESAREA_CVE,TIPO_CESAREA_STR,PERSONAL_ATENDIO_CVE,PERSONA_ATENDIO_STR,MEDICO_ATENDIO_CVE,MEDICO_ATENDIO_STR,SOBREVIVIO_PARTO_CVE,SOBREVIVIO_PARTO_STR,FECHA_NACIMIENTO_MADRE,ES_GINECO
0,PLIMB004131,21,PUEBLA,1,Eutócico (natural),8,NO APLICA,1,MÉDICO,14.0,MÉDICO GENERAL,1,SI,16/08/2005,False
1,PLIMB004131,21,PUEBLA,1,Eutócico (natural),8,NO APLICA,1,MÉDICO,14.0,MÉDICO GENERAL,1,SI,20/01/1995,False
2,PLIMB004131,21,PUEBLA,1,Eutócico (natural),8,NO APLICA,1,MÉDICO,14.0,MÉDICO GENERAL,1,SI,27/05/2002,False
3,PLIMB004131,21,PUEBLA,1,Eutócico (natural),8,NO APLICA,1,MÉDICO,14.0,MÉDICO GENERAL,1,SI,22/09/1995,False
4,SRIMB001766,26,SONORA,3,Cesárea,2,DE URGENCIA,1,MÉDICO,11.0,MÉDICO GINECO OBSTETRA,1,SI,29/10/1994,True


In [2278]:
#Función para agrupar por clues. 
df_top_ranking_hospitales = (
    df_top_ranking_hospitales.groupby('CLUES')
    .agg(
        TOTAL_PARTOS=('TIPO_PARTO_STR', 'count'),
        PARTOS_CON_GINECO=('ES_GINECO', 'sum')
    )
    .reset_index()
)

df_top_ranking_hospitales['PARTOS_SIN_GINECO'] = df_top_ranking_hospitales['TOTAL_PARTOS'] - df_top_ranking_hospitales['PARTOS_CON_GINECO']

df_top_ranking_hospitales

,CLUES,TOTAL_PARTOS,PARTOS_CON_GINECO,PARTOS_SIN_GINECO
0,ASIMS000016,1564,1434,130
1,ASIMS000021,1659,1630,29
2,ASIMS000161,1619,1519,100
3,ASIST000016,139,113,26
4,ASSMP000016,55,54,1
...,...,...,...,...
5093,ZSSSA002136,31,22,9
5094,ZSSSA002310,110,25,85
5095,ZSSSA002322,151,99,52
5096,ZSSSA012450,2482,722,1760


**Pegar metadatos**

In [2315]:
df_tabla_hospitales = df_top_ranking_hospitales[[
    'ENTIDAD',
    'NOMBRE COMERCIAL',
    'TOTAL_PARTOS',
    'PARTOS_SIN_GINECO',
    'TASA_SIN_GINECO',
    'NIVEL ATENCION'
]]

In [2316]:
# 2. Ordenamos por partos sin gineco y nos quedamos con el top 10
df_tabla_hospitales = df_tabla_hospitales.sort_values(
    by='PARTOS_SIN_GINECO', ascending=False
).head(10)

In [2317]:
# 3. Renombramos las columnas al inglés para Datawrapper
df_tabla_hospitales = df_tabla_hospitales.rename(columns={
    'ENTIDAD': 'State',
    'NOMBRE COMERCIAL': 'Hospital Name',
    'TOTAL_PARTOS': 'Total Births',
    'PARTOS_SIN_GINECO': 'Births Without OB-GYN',
    'TASA_SIN_GINECO': 'Percent Without OB-GYN',
    'NIVEL ATENCION': 'Level of Care'
})

In [2318]:
df_tabla_hospitales.to_csv("top_10_hospitals_obgyn_gap_en.csv", index=False)


In [2319]:
df_tabla_hospitales

,State,Hospital Name,Total Births,Births Without OB-GYN,Percent Without OB-GYN,Level of Care
1138,CIUDAD DE MEXICO,HOSPITAL DE LA MUJER,3835,3833,99.95,TERCER NIVEL
1330,GUERRERO,HOSPITAL DE LA MADRE Y DEL NIÑO DE TLAPA,2946,2050,69.59,SEGUNDO NIVEL
3661,PUEBLA,CLÍNICA MATERNO INFANTIL,2088,1888,90.42,SEGUNDO NIVEL
4018,SINALOA,HOSPITAL DE LA MUJER,2051,1818,88.64,TERCER NIVEL
5096,ZACATECAS,HOSPITAL DE LA MUJER,2482,1760,70.91,SEGUNDO NIVEL
4104,SINALOA,HOSPITAL DE LA MUJER,2351,1756,74.69,SEGUNDO NIVEL
3288,OAXACA,MIAHUATLÁN DE PORFIRIO DÍAZ,2544,1726,67.85,SEGUNDO NIVEL
27,AGUASCALIENTES,HOSPITAL DE LA MUJER,4918,1724,35.05,TERCER NIVEL
1723,GUANAJUATO,HOSPITAL MATERNO CELAYA,3143,1714,54.53,TERCER NIVEL
3168,OAXACA,HG OAXACA DR. AURELIO VALDIVIESO,2190,1708,77.99,SEGUNDO NIVEL


In [2323]:
df_tabla_hospitales["Percent Without OB-GYN"] = df_tabla_hospitales["Percent Without OB-GYN"].apply(lambda x: f"{x:.2f}")

In [2324]:
df_tabla_hospitales.to_csv("top_10_hospitals_obgyn_gap_en.csv", index=False)

# Tipo de cesáreas en México

## Análisis de tipos de cesárea en México (2024)

A partir del DataFrame limpio (`df_nacimientos_2024_clean_clues`), vamos a:
1. Filtrar sólo los partos que fueron cesáreas  
2. Contar todas las categorías de cesárea  
3. Construir un DataFrame con totales y porcentajes  
4. Exportar el CSV 

**Filtrar solo los partos que fueron cesáreas**

In [2286]:
df_ces_clean = df_nacimientos_2024_clean_clues[
  df_nacimientos_2024_clean_clues['TIPO_PARTO_STR']=='Cesárea'
]

**Contar todas las categorías de cesárea (incluye NO ESPECIFICADO, NO APLICA…)**

In [2287]:
counts_clean = (
  df_ces_clean['TIPO_CESAREA_CVE']
    .map(cesareas_map)
    .value_counts()
)

**Construir el DataFrame final con totales y porcentaje**


In [2288]:
pct_urg_clean = round(counts_clean['DE URGENCIA'] / counts_clean.sum() * 100, 1)
print(f"{pct_urg_clean}% de las cesáreas fueron de urgencia.")

52.1% de las cesáreas fueron de urgencia.


In [2289]:
df_ces_clean = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['TIPO_PARTO_STR']=='Cesárea'
]

In [2290]:
counts_clean = (
    df_ces_clean['TIPO_CESAREA_CVE']
    .map(cesareas_map)
    .value_counts()
)

**Este CSV tendrá columnas `TIPO_CESAREA`, `TOTAL_CESAREAS` y `PORCENTAJE`**


In [2291]:
df_plot = counts_clean.rename_axis('TIPO_CESAREA') \
                      .reset_index(name='TOTAL_CESAREAS')
df_plot['PORCENTAJE'] = (
    df_plot['TOTAL_CESAREAS'] / df_plot['TOTAL_CESAREAS'].sum() * 100
).round(1)


In [2292]:
df_plot

,TIPO_CESAREA,TOTAL_CESAREAS,PORCENTAJE
0,DE URGENCIA,390355,52.1
1,PROGRAMADA,328179,43.8
2,NO ESPECIFICADO,21000,2.8
3,NO APLICA,9462,1.3


**Guardar CSV listo para plot**

In [2293]:
df_plot.to_csv('tipos_cesareas_mexico_2024_plot.csv', index=False)

## MORTALIDAD MATERNA

**¿Cuántos partos en df_nacimientos_2024 resultaron en la muerte de la madre?**

**Extraer sola las columnas pertinentes para el análisis de muertes maternas, haciendo una copia de `df_nacimientos_2024_clean_clues`, para no modificar el df original**


In [2294]:
df_mortalidad_materna_2024 = df_nacimientos_2024_clean_clues[[
    'CLUES',
    'SOBREVIVIO_PARTO_CVE',
    'SOBREVIVIO_PARTO_STR',
    'ENTIDAD_PARTO_STR'
]].copy()   

In [2295]:
df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] = (
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE']
    .astype(str)
)

**Filtrar los casos de muerte materna (SOBREVIVIO_PARTO_CVE == '2')**


In [2296]:
df_madre_no_sobrevivio = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '2'
]

**Filtrar los casos donde la madre sobrevivió (SOBREVIVIO_PARTO_CVE == '1')**

In [2297]:
df_madre_sobrevivio = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '1'
]

**Filtrar los casos “no especificado” (SOBREVIVIO_PARTO_CVE == '0')**

In [2298]:
df_madre_no_especificado = df_mortalidad_materna_2024[
    df_mortalidad_materna_2024['SOBREVIVIO_PARTO_CVE'] == '0'
]

**Resultados de la operación sobre supervivencia de las madres**

In [2299]:
print(f"Según la base de datos de nacimientos, solo se reportaron {len(df_madre_no_sobrevivio)} muertes maternas en 2024")
print(f"Se reportó que en {len(df_madre_sobrevivio)} partos, la madre sobrevivió.")
print(f"Hay {len(df_madre_no_especificado)} donde no se especifica si la madre sobrevivió o no al trabajo de parto")


Según la base de datos de nacimientos, solo se reportaron 76 muertes maternas en 2024
Se reportó que en 1338444 partos, la madre sobrevivió.
Hay 385 donde no se especifica si la madre sobrevivió o no al trabajo de parto


## ¿Qué tipo de medico las atendió? 


Para asegurarnos de que estamos contando **todas** las muertes (incluyendo aquellas sin etiqueta de médico), seguimos estos pasos:

1. **Filtrar** las muertes maternas (`SOBREVIVIO_PARTO_CVE == 2`) sobre el DataFrame limpio.  
2. **Contar** por código (`MEDICO_ATENDIO_CVE`) y **mapear** a etiquetas legibles.  
3. Incluir explícitamente los `NaN` con `dropna=False` para no perder ningún caso.  
4. **Calcular** el porcentaje sobre el total de muertes.  
5. **Verificar** que la suma de `cantidad` coincida con el número de muertes y que los porcentajes sumen 100 %.  

**Filtrar muertes maternas (código entero 2)**

In [2300]:
df_madre_no_sobrevivio = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues["SOBREVIVIO_PARTO_CVE"] == 2
]

**Contar por código y luego mapear etiquetas (incluye NaN)**

In [2301]:
conteo_medicos = (
    df_madre_no_sobrevivio["MEDICO_ATENDIO_CVE"]
    .map(medico_map)
    .value_counts(dropna=False)      # incluye los NaN
    .rename_axis("TIPO_MEDICO")
    .reset_index(name="cantidad")
)

**Calcular porcentaje**

In [2302]:
total = conteo_medicos["cantidad"].sum()
conteo_medicos["porcentaje"] = (conteo_medicos["cantidad"] / total * 100).round(1)

In [2303]:
conteo_medicos

,TIPO_MEDICO,cantidad,porcentaje
0,MÉDICO GINECO OBSTETRA,56,73.7
1,MÉDICO GENERAL,11,14.5
2,OTRO ESPECIALISTA,3,3.9
3,MIP,3,3.9
4,RESIDENTE,2,2.6
5,NaN,1,1.3


In [2304]:
conteo_medicos.to_csv("tipos_medico_muerte_materna_plot.csv", index=False)

# ¿En qué hospitales hubo más muertes maternas? 

In [2305]:
#VERIFICAR LOS UNIQUE EN EL DF DE MUERTES MATERNAS

hospitales_muertes_maternas = df_madre_no_sobrevivio['CLUES'].unique()

#CONTAR LOS REPETIDOS

df_clues_repetidos = df_madre_no_sobrevivio['CLUES'].value_counts()

#CONVERTIR DE SERIE A DF PARA PODER COMPARAR EN EL FUTURO

df_clues_repetidos = df_clues_repetidos.to_frame()

print(type(df_clues_repetidos))

df_clues_con_nombre = pd.merge(
    df_clues_repetidos,
    df_establecimientos_salud_2024[['CLUES', 'NOMBRE DE LA UNIDAD','NOMBRE DE LA INS ADM','ENTIDAD', 'LATITUD', 'LONGITUD']],
    on='CLUES',
    how='left'
)

df_clues_con_nombre.head()


<class 'pandas.core.frame.DataFrame'>


,CLUES,count,NOMBRE DE LA UNIDAD,NOMBRE DE LA INS ADM,ENTIDAD,LATITUD,LONGITUD
0,ZSSSA012450,3,HOSPITAL DE LA MUJER,SERVICIOS DE SALUD IMSS BIENESTAR,ZACATECAS,22.744225,-102.500206
1,DFIST000534,2,HG TACUBA,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES PA...,CIUDAD DE MEXICO,19.453792,-99.190950
2,VZIST000120,2,CH XALAPA,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES PA...,VERACRUZ DE IGNACIO DE LA LLAVE,19.545196,-96.937455
3,CHSSA000676,2,HOSPITAL CENTRAL DEL ESTADO,SECRETARIA DE SALUD ESTATAL,CHIHUAHUA,28.644160,-106.064920
4,DFSSA003973,2,HOSPITAL GENERAL DE MÉXICO “DR. EDUARDO LICEAGA”,HOSPITAL FEDERAL DE REFERENCIA,CIUDAD DE MEXICO,19.414719,-99.115105


In [2306]:
#Generar .csv de los hospitales con más de 2 registros con muerte materna
df_semaforo_gineco.to_csv('hospitales_muertes_materna.csv', index=False)

**En la base de nacimientos_2024 se reportaron 76 muertes maternas y otras 385 muertes maternas clasificadas como “no especificado”. Es necesario indagar en qué hospitales ocurrieron estos casos para comprender mejor las condiciones de atención y los vacíos en el registro.**

In [2307]:
def clasificar_parto_detallado(tipo_parto_str, tipo_cesarea_str):
    if tipo_parto_str == "Cesárea":
        if tipo_cesarea_str == "PROGRAMADA":
            return "Cesárea programada"
        elif tipo_cesarea_str == "DE URGENCIA":
            return "Cesárea de urgencia"
        else:
            return "Cesárea no especificada"
    elif tipo_parto_str == "Eutócico (natural)":
        return "Eutócico - Natural"
    elif tipo_parto_str == "Distócico (natural complicado)":
        return "Distócico - Natural Complicao"
    elif tipo_parto_str == "Otro":
        return "Otro"
    else:
        return "Parto No especificado"


In [2308]:
df_nacimientos_2024_clean_clues["PARTO_DETALLADO"] = df_nacimientos_2024_clean_clues.apply(
    lambda row: clasificar_parto_detallado(row["TIPO_PARTO_STR"], row["TIPO_CESAREA_STR"]),
    axis=1
)

In [2309]:
df_nacimientos_2024_clean_clues["PARTO_DETALLADO"].value_counts()

PARTO_DETALLADO
Eutócico - Natural               575328
Cesárea de urgencia              390355
Cesárea programada               328179
Cesárea no especificada           30462
Distócico - Natural Complicao     12390
Parto No especificado              1180
Otro                               1011
Name: count, dtype: int64

In [2310]:
df_muertes_detallado = df_nacimientos_2024_clean_clues[
    df_nacimientos_2024_clean_clues['SOBREVIVIO_PARTO_CVE'] == 2
].copy()

### ¿En qué tipo de parto murieron más mujeres?
De las 76 muertes maternas registradas en la base de nacimientos 2024, **casi la mitad** ocurrieron tras una **cesárea de urgencia** (48.7 %), el tipo de parto con mayor letalidad registrada.

El 30.3 % de las muertes ocurrieron tras un parto **eutócico natural**, y un 17.1 % luego de una **cesárea programada**, lo que indica que incluso los procedimientos no emergentes pueden implicar riesgos si no se realizan bajo condiciones adecuadas.

Solo un caso se reportó como “cesárea no especificada”, lo cual también alerta sobre **inconsistencias o vacíos en el registro** de la atención obstétrica.


In [2311]:
conteo_muertes = (
    df_muertes_detallado["PARTO_DETALLADO"]
    .value_counts()
    .rename_axis("TIPO_PARTO_MUERTE_MATERNA")
    .reset_index(name="Cantidad")
)

In [2312]:
total_muertes = conteo_muertes["Cantidad"].sum()
conteo_muertes["Porcentaje"] = round((conteo_muertes["Cantidad"] / total_muertes) * 100, 1)

In [2313]:
conteo_muertes

,TIPO_PARTO_MUERTE_MATERNA,Cantidad,Porcentaje
0,Cesárea de urgencia,37,48.7
1,Eutócico - Natural,23,30.3
2,Cesárea programada,13,17.1
3,Distócico - Natural Complicao,2,2.6
4,Cesárea no especificada,1,1.3


In [2314]:
conteo_muertes.to_csv("muertes_maternas_por_parto_detallado.csv", index=False)